In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print('GPU state:', device)

GPU state: cuda:0


In [2]:
# Cifar-10 data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


# Data
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
trainLoader = torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=True, num_workers=2)
testLoader = torch.utils.data.DataLoader(testset, batch_size=8, shuffle=False, num_workers=2)

# Data classes
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# Model structure
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net().to(device)
print(net)


# Parameters
criterion = nn.CrossEntropyLoss()
lr = 0.001
epochs = 3
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
# Train
for epoch in range(epochs):
    running_loss = 0.0

    for times, data in enumerate(trainLoader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

        if times % 100 == 99 or times+1 == len(trainLoader):
            print('[%d/%d, %d/%d] loss: %.3f' % (epoch+1, epochs, times+1, len(trainLoader), running_loss/2000))

print('Finished Training')

[1/3, 100/6250] loss: 0.115
[1/3, 200/6250] loss: 0.230
[1/3, 300/6250] loss: 0.345
[1/3, 400/6250] loss: 0.460
[1/3, 500/6250] loss: 0.575
[1/3, 600/6250] loss: 0.689
[1/3, 700/6250] loss: 0.803
[1/3, 800/6250] loss: 0.916
[1/3, 900/6250] loss: 1.028
[1/3, 1000/6250] loss: 1.136
[1/3, 1100/6250] loss: 1.242
[1/3, 1200/6250] loss: 1.344
[1/3, 1300/6250] loss: 1.445
[1/3, 1400/6250] loss: 1.544
[1/3, 1500/6250] loss: 1.644
[1/3, 1600/6250] loss: 1.742
[1/3, 1700/6250] loss: 1.839
[1/3, 1800/6250] loss: 1.936
[1/3, 1900/6250] loss: 2.030
[1/3, 2000/6250] loss: 2.124
[1/3, 2100/6250] loss: 2.218
[1/3, 2200/6250] loss: 2.312
[1/3, 2300/6250] loss: 2.404
[1/3, 2400/6250] loss: 2.494
[1/3, 2500/6250] loss: 2.586
[1/3, 2600/6250] loss: 2.675
[1/3, 2700/6250] loss: 2.763
[1/3, 2800/6250] loss: 2.853
[1/3, 2900/6250] loss: 2.939
[1/3, 3000/6250] loss: 3.027
[1/3, 3100/6250] loss: 3.112
[1/3, 3200/6250] loss: 3.199
[1/3, 3300/6250] loss: 3.287
[1/3, 3400/6250] loss: 3.373
[1/3, 3500/6250] loss: 

In [5]:
# Test
correct = 0
total = 0
with torch.no_grad():
    for data in testLoader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test inputs: %d %%' % (100 * correct / total))

Accuracy of the network on the 10000 test inputs: 57 %
